In [ ]:
from qiling.core import *
from qiling.const import *
from qiling.os.const import *
from unicorn.arm_const import *
from unicorn import *
import math


In [ ]:
filepath='stm32f103RCT6.bin'
with open(filepath, 'rb') as fp:
    fw = fp.read()

In [ ]:
# 加载地址
load_addr=0x8000000 

# 栈大小
stack_size = 0x20000 

# 堆大小
heap_size = 0x20000 

# 计算固件大小 0x1000对齐
fw_size = math.ceil(len(fw)/0x1000)*0x1000 

# 初始分配的内存大小包括 固件和栈空间大小，+0x1000是为了使可用的栈大小与stack_size保持一致
ram_size = fw_size + stack_size + 0x1000 

cfg_str = f"""
[CODE]
ram_size = {ram_size}
entry_point = {load_addr}
heap_size = {heap_size}


[MISC]
current_path = /
"""

# 保存配置文件
with open('ql-config.ql', 'w') as fp:
    fp.write(cfg_str)

In [ ]:
ql = Qiling(code=fw, archtype="arm", ostype="blob", profile="ql-config.ql", thumb=True)

In [ ]:
begin =0x800E288
end = 0x800E296

In [ ]:
sp = ql.arch.regs.sp

uc=ql.uc

#为第1个参数key传参
uc.reg_write(UC_ARM_REG_R0, sp+0x100) 
uc.mem_write(sp+0x100, bytes.fromhex("BA 2F 96 A9 BA 2F 96 A9 BA 2F 96 A9 BA 2F 96 A9"))

#为第2个参数in传参
uc.reg_write(UC_ARM_REG_R1, sp+0x200)
uc.mem_write(sp+0x200, bytes.fromhex("10 BE 62 F8 E8 DC 34 46"))

#为第3个参数out传参
uc.reg_write(UC_ARM_REG_R2, sp+0x300)


In [ ]:
ql.debugger = 'gdb:0.0.0.0:9999'

In [ ]:
## qiling-1.4.4中需要用到这部分代码，qiling后续版本如果有相应实现，就不在需要这部分代码了
class MyMapper:
    def add_fs_mapping(self, ql_path, real_dest):
        pass
    
ql.os.fs_mapper = MyMapper()
## qiling-1.4.4中需要用到这部分代码，qiling后续版本如果有相应实现，就不在需要这部分代码了

### Thumb 断点问题 - 开始
- qiling-1.4.4中，模拟运行thumb指令后，IDA下断点，停不下来，原因是IDA下的断点地址没有+1，qiling运行时根据thumb自动将address+1，导致两者不相等
- 可以用下面代码块动态解决这个问题，后续qiling如果修复这个问题，这部分代码就不需要了

In [ ]:
import types

In [ ]:
def dbg_hook(self, ql, address: int, size: int):
    #if ql.arch.type == QL_ARCH.ARM and ql.arch.is_thumb:
    #        address += 1
    
    # resuming emulation after hitting a breakpoint will re-enter this hook.
    # avoid an endless hooking loop by detecting and skipping this case
    if address == self.last_bp:
        self.last_bp = None

    elif address in self.bp_list:
        self.last_bp = address

        ql.log.info(f'gdb> breakpoint hit, stopped at {address:#x}')
        ql.stop()

    # # TODO: not sure what this is about
    # if address + size == self.exit_point:
    #     ql.log.debug(f'{PROMPT} emulation entrypoint at {self.entry_point:#x}')
    #     ql.log.debug(f'{PROMPT} emulation exitpoint at {self.exit_point:#x}')

In [ ]:
def run(self, begin: Optional[int] = None, end: Optional[int] = None, timeout: int = 0, count: int = 0):
        """Start binary emulation.

        Args:
            begin   : emulation starting address
            end     : emulation ending address
            timeout : limit emulation to a specific amount of time (microseconds); unlimited by default
            count   : limit emulation to a specific amount of instructions; unlimited by default
        """

        # replace the original entry point, exit point, timeout and count
        self.entry_point = begin
        self.exit_point = end
        self.timeout = timeout
        self.count = count

        # init debugger (if set)
        debugger = select_debugger(self._debugger)

        if debugger:
            debugger = debugger(self)
            debugger.gdb.dbg_hook = types.MethodType(dbg_hook, debugger.gdb)

        # patch binary
        self.do_bin_patch()

        if self.baremetal:
            if self.count <= 0:
                self.count = -1

            self.arch.run(count=self.count, end=self.exit_point)
        else:
            self.write_exit_trap()
            # emulate the binary
            self.os.run()

        # run debugger
        if debugger and self.debugger:
            debugger.run()

In [ ]:
ql.run=types.MethodType(run, ql)

### Thumb 断点问题 - 结束

In [ ]:
ql.run(begin, end)

In [ ]:
ql.mem.read(sp+0x300, 8).hex()